In [ ]:
!pwd

In [35]:
%load_ext autoreload
%autoreload 2

import sys, os
import logging, typing
from torchvision import transforms

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import itertools

import torch
from tqdm import tqdm

sys.path.append('../util')
import panoseti_file_interfaces as pfi
import pff
from vae_model import *
from ph_dataset import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
x = torch.arange(5 * 256).reshape(5, 16, 16) * 1.0

In [32]:
x.amax((1, 2)).shape

torch.Size([5])

In [36]:
def show_dataset_sample(dataset, nrows=5, ncols=15, cmap='rocket', log_cbar=False, norm=False):
    """Preview images from the dataset."""
    nsamples = nrows * ncols
    idxs = np.arange(len(dataset))
    sample_idxs = np.random.choice(idxs, nsamples, replace=False)
    
    if len(sample_idxs) < nsamples:
        print('Not enough training data')

    f = plt.figure(figsize=(ncols, nrows))
    # f.tight_layout()

    canvas = np.zeros((nrows * 16, ncols * 16))
    xy_indices = itertools.product(range(nrows), range(ncols))
    for i, (row, col) in enumerate(xy_indices):
        idx = sample_idxs[i]
        canvas[row*16:(row+1)*16, col*16:(col+1)*16] = ph_dataset.get_ph_data(idx)['img']
    if norm:
        canvas = canvas
    if log_cbar:
        img_plt = plt.imshow(canvas, cmap=cmap)
        # format cbar
        cbar = plt.colorbar(img_plt, fraction=0.027, pad=0.025)
        cbar.ax.yaxis.set_major_formatter(FuncFormatter(colorbar_formatter))
        plt.title("Log-Normalized Real Pulse Height Images")
    else:
        img_plt = plt.imshow(canvas, cmap=cmap)
        # format cbar
        cbar = plt.colorbar(img_plt, fraction=0.027, pad=0.025)
        plt.title("Real Pulse Height Images")
    plt.show(f)

In [37]:
ph_dataset_config = {
    "max_ph_frames": 10_000,
    "observing_runs": [
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2023-07-14T04:46:21Z.runtype_sci-obs.pffd',
            "module_ids": 'all',
        },
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2023-08-15T07:15:50Z.runtype_sci-obs.pffd',
            "module_ids": 'all',
        },
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2023-09-14T04:25:43Z.runtype_sci-obs.pffd',
            "module_ids": 'all',
        },
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2023-10-12T04:28:34Z.runtype_eng-test.pffd',
            "module_ids": 'all',
        },
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2024-05-24T04:37:30Z.runtype_sci-data.pffd',
            "module_ids": 'all',
        },
        {
            "data_dir": './test_data',
            "run_dir": 'obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd',
            "module_ids": 'all',
        },
    ]
}

In [39]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ConvertImageDtype(torch.float),
])

ph_dataset = PulseHeightDataset(ph_dataset_config, transform=transform, log_level=logging.INFO)
ph_dataset.reset_ph_generator()

Computing PH baselines for obs_Lick.start_2023-07-14T04:46:21Z.runtype_sci-obs.pffd


100%|███████████████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:00<00:00, 14486.43frames/s]


Computing PH baselines for obs_Lick.start_2023-08-15T07:15:50Z.runtype_sci-obs.pffd


100%|███████████████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:00<00:00, 14393.80frames/s]


Computing PH baselines for obs_Lick.start_2023-09-14T04:25:43Z.runtype_sci-obs.pffd


...

Computing PH baselines for obs_Lick.start_2024-07-25T04:34:06Z.runtype_sci-data.pffd


100%|███████████████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:00<00:00, 13114.65frames/s]
[INFO][2025-01-23:17:50:44,308][ph_dataset.py:189] Init: dataset_length = 10000
[INFO][2025-01-23:17:50:44,308][ph_dataset.py:189] Init: dataset_length = 10000
[INFO][2025-01-23:17:50:44,308][ph_dataset.py:196] Init: min_max_pixel_z_score = 5
[INFO][2025-01-23:17:50:44,308][ph_dataset.py:196] Init: min_max_pixel_z_score = 5


Computing PH image statistics


100%|███████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4606.61ph_frames/s]
[INFO][2025-01-23:17:50:46,575][ph_dataset.py:322] Resetting dataset-global PH frame generator
[INFO][2025-01-23:17:50:46,575][ph_dataset.py:322] Resetting dataset-global PH frame generator
[INFO][2025-01-23:17:50:46,575][ph_dataset.py:322] Resetting dataset-global PH frame generator
[INFO][2025-01-23:17:50:46,575][ph_dataset.py:322] Resetting dataset-global PH frame generator


{'mean': array(5.6854563, dtype=float32), 'median': array(0., dtype=float32), 'std': array(43.793335, dtype=float32), 'min': array(-174., dtype=float32), 'max': array(3286., dtype=float32), 'norm_channel_max': array(170.6745, dtype=float32), 'q9995': tensor(709.)}


In [ ]:
# PH data EDA: visualize PH image and the distribution of pixel values.
# ph_dataset.reset_ph_generator()
ph_data = ph_dataset.get_ph_data(2)

assert ph_data['img'] is not None
f, axs = plt.subplots(1, 2, figsize=(8 * 4, 6))
ph_dataset.plot_ph_img(ph_data['img'], ph_data['meta'], ax=axs[0])
ph_dataset.plot_ph_pixel_dist(ph_data['img'],  ph_data['meta'], ax=axs[1], upper=None)
# ph_dataset.plot_ph_pixel_dist(ph_data['img'],  ph_data['meta'], ax=axs[2], upper=False)
# ph_dataset.plot_ph_pixel_dist(ph_data['img'],  ph_data['meta'], ax=axs[3], upper=True)
f.suptitle(f"16x16 11.5 PE Pulse-Height Image at {ph_data['meta']['unix_timestamp']}")
plt.show()
ph_data['meta']

In [ ]:
show_dataset_sample(ph_dataset, nrows=15, ncols=25, norm=True)